In [0]:
import os
from pyspark.sql.functions import col, expr, substring,split,regexp_replace
from pyspark.sql.functions import sum,round
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, lpad, lit

In [0]:
df_car_register=spark.read.option("header", "true").csv('/Volumes/project/default/project/car_register_cleaned_csv/')

df_Activate_GreenRatio=spark.read.option("header", "true").option("inferSchema", "true").csv('/Volumes/project/default/project/Activate_GreenRatio_edit.csv/')

df_Building_Density=spark.read.option("header", "true").option("inferSchema", "true").csv('/Volumes/project/default/project/Building_Density.csv')

df_population_density=spark.read.option("header", "true").option("inferSchema", "true").csv('/Volumes/project/default/project/population_density_edit.csv/')

df_seoul_region_driving_with_road=spark.read.option("header", "true").option("inferSchema", "true").csv('/Volumes/project/default/project/seoul_region_driving_with_road_edit.csv/')

#df_sigungu_code=spark.read.option("header", "true").option("inferSchema", "true").csv('/Volumes/project/default/project/sigungu-code.csv')

#df_s_dot_humid_2023=spark.read.option("header", "true").option("inferSchema", "true").csv('/Volumes/project/default/project/s-dot_humid_2023.csv')
                                                                                                        

In [0]:
display(df_Activate_GreenRatio)
display(df_Building_Density)
display(df_car_register)
display(df_population_density)
display(df_seoul_region_driving_with_road)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
df_seoul_region_driving_with_road2 = df_seoul_region_driving_with_road.filter(col('year') == 2023)
df_seoul_region_driving_with_road2=df_seoul_region_driving_with_road2.withColumn("year",lit(2024))
df_seoul_region_driving_with_road=df_seoul_region_driving_with_road.union(df_seoul_region_driving_with_road2)

join1

In [0]:
df_Building_Density=df_Building_Density.withColumnRenamed("District","district2")
df_j1=df_Activate_GreenRatio.join(df_Building_Density,df_Activate_GreenRatio.District==df_Building_Density.district2)

In [0]:
df_j1=df_j1.drop("district2","area_m2","dis_area_m2","Total_Floor_Area","Area")
display(df_j1)

전처리

In [0]:
df=df_car_register
df = df.withColumn('year', split(col('year_month'), '-').getItem(0))
df = df.withColumn('month', split(col('year_month'), '-').getItem(1))
display(df)

join2

In [0]:
df_car_register=df.withColumnRenamed("district","district2")
df_car_register=df_car_register.withColumnRenamed("year","year2")
df_j2=df_j1.join(df_car_register,
                 (df_j1.District==df_car_register.district2) & (df_j1.year==df_car_register.year2))
df_j2=df_j2.drop("district2","year2","year_month")
display(df_j2)

join3

In [0]:
df=df_population_density.filter(df_population_density.year>=2020)
df=df.withColumnRenamed("District","district2")
df=df.withColumnRenamed("year","year2")
display(df)

In [0]:
df_j3=df_j2.join(df,
                 (df_j2.District==df.district2) & (df_j2.year==df.year2))
df_j3=df_j3.drop("district2","year2","total_population","total_area")
display(df_j3)

join4

In [0]:
df=df_seoul_region_driving_with_road.withColumnRenamed("District","district2")
df=df.withColumnRenamed("year","year2")
df_j4=df_j3.join(df,
                 (df_j3.District==df.district2) & (df_j3.year==df.year2))
df_j4=df_j4.drop("district2","year2")
display(df_j4)


In [0]:
df=df_j4

In [0]:
# output_path = "/Volumes/project/default/project/join_v1.csv"
# df.coalesce(1).write.mode("overwrite").option("header", "true").csv(output_path)

# print(f"DataFrame이 '{output_path}' 경로에 성공적으로 덮어쓰여졌습니다.")

In [0]:
df.groupBy("year").count().orderBy("year").show()